In [2]:
特征：
   'label','popularity','carCommentVolum','newsReplyVolum'平移了12个月
    train:
        1,  2,  3,  4,  5,  6,  7,  8,  9,  10,  11,  12
        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,  23,  24 
        25, 26, 27, 28
        
    test: 21,22,23,24

SyntaxError: invalid character in identifier (<ipython-input-2-027fb5b461bb>, line 1)

In [1]:
# import sys
import numpy as np
import pandas as pd
# import os 
# import gc
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
import datetime
import time
import lightgbm as lgb
import xgboost as xgb
import warnings
from sklearn.cluster import KMeans
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [22]:
# path  = './ccf_car/'
train_sales  = pd.read_csv('train_sales_data.csv')


train_search = pd.read_csv('train_search_data.csv')
train_user   = pd.read_csv('train_user_reply_data.csv')
evaluation_public = pd.read_csv('evaluation_public.csv')
# submit_example    = pd.read_csv('submit_example.csv')
data = pd.concat([train_sales, evaluation_public], ignore_index=True)
data = data.merge(train_search, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user, 'left', on=['model', 'regYear', 'regMonth'])
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])
#索引-->value
#LabelEncoder
for i in ['bodyType', 'model']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))
    #key-->value
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']
initial_key_list = ['adcode','bodyType','forecastVolum','id','model','province','regMonth','regYear','salesVolume','popularity','carCommentVolum','newsReplyVolum'	,'label','mt']
data = data[initial_key_list].copy()
data['pred_label'] = 0
#去掉12月
data = data[~(data['mt']==12)]
data = data[~(data['mt']==24)]
data.tail()

,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum,label,mt,pred_label
36955,350000,0,NaN,5364,59,福建,4,2018,NaN,NaN,NaN,NaN,NaN,28,0
36956,210000,0,NaN,5365,59,辽宁,4,2018,NaN,NaN,NaN,NaN,NaN,28,0
36957,500000,0,NaN,5366,59,重庆,4,2018,NaN,NaN,NaN,NaN,NaN,28,0
36958,610000,0,NaN,5367,59,陕西,4,2018,NaN,NaN,NaN,NaN,NaN,28,0
36959,230000,0,NaN,5368,59,黑龙江,4,2018,NaN,NaN,NaN,NaN,NaN,28,0


In [23]:
def Cluster(df_,pred_month): 
    df_['cluster_label']  =-1
    df = df_[df_['mt']<pred_month]
    df= df[['adcode','model','mt','label']].set_index(['model','adcode','mt']).reset_index()
    kmeans = KMeans(n_clusters = 3) 
    kmeans.fit(df.values)
    df_.loc[df_['mt']<pred_month,'cluster_label'] =  kmeans.labels_
#     print(df.head())
#     df = df.merge(df,'left',on = ['adcode','model','mt','label'])
    return df_




In [24]:
def windows_feature(data):
#     if (judge >1):
    print("windows_var_features start")
    df = data.copy()
    df_all = pd.DataFrame()
    df['model_adcode'] = df['adcode'] + df['model']
    model_adcode = list(df.model_adcode.unique())
    for i in tqdm(model_adcode):
        idx = (df['model_adcode']==i)
        df2 = df[idx] 

        df2['windows3_var'] = df2['label'].rolling(3).var()
        df2['windows5_var'] = df2['label'].rolling(5).var()
        df2['windows7_var'] = df2['label'].rolling(7).var()

        df2['windows3_var'] = df2['windows3_var'].rolling(3).var()
        df2['windows5_var'] = df2['windows5_var'].rolling(5).var()
        df2['windows7_var'] = df2['windows7_var'].rolling(7).var()

        df_all = pd.concat([df_all, df2], ignore_index=True)
    data = data.merge(df_all, 'left', on=['cluster_label','adcode','bodyType','forecastVolum','id','model','province','regMonth','regYear','salesVolume','popularity','carCommentVolum','newsReplyVolum','pred_label','label','mt'])
    print("windows_var_features end")
    return data

        

In [25]:
def get_stat_feature(df_):   
    df = df_.copy()
    stat_feat = []
    df['model_adcode'] = df['adcode'] + df['model']
    df['model_adcode_mt'] = df['model_adcode'] * 100 + df['mt']
    for col in tqdm(['label','popularity','newsReplyVolum']):#,'newsReplyVolum'
    # shift
        for i in [1,2,3,4,5,6,7,8,9,10,11]:
            stat_feat.append('shift_model_adcode_mt_{}_{}'.format(col,i))
            df['model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'] + i
            df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,i))
            df['shift_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])
# 
#     for i in [1]: #本月销量和前1-3,12个月相减
#         df["differ_{}_{}".format(col,i)] = df['{}'.format(col)] -df['shift_model_adcode_mt_{}_{}'.format(col,i)]
#         df["differ2_{}_{}".format(col,i)] = df["differ_{}_{}".format(col,i)]-df.groupby('model_adcode')["differ_{}_{}".format(col,i)].shift(1)
#         stat_feat.append("differ2_{}_{}".format(col,i))

#         df["ratio_{}_{}".format(col,i)] = df['{}'.format(col)] /df['shift_model_adcode_mt_{}_{}'.format(col,i)]
#         df["ratio2_{}_{}".format(col,i)] = df["ratio_{}_{}".format(col,i)]/df.groupby('model_adcode')["ratio_{}_{}".format(col,i)].shift(1)
#         stat_feat.append("ratio2_{}_{}".format(col,i))
    return df,stat_feat

In [26]:
def get_stat_feature2(df_):  #bodytype+省份
    df = df_.copy()
    print(df.shape)
    stat2_feat = []
    df['body_adcode'] = df['adcode'] + df['bodyType']
    df['body_adcode_mt'] = df['body_adcode'] * 100 + df['mt']
    df2 = df.copy()
    for col in tqdm(['label','popularity','newsReplyVolum']):
        df = df.groupby(['bodyType','adcode','body_adcode','mt','body_adcode_mt'])[col].mean().reset_index()
        for i in [1,2,3,4,5,6,7,8,9,10,11]:
            df['shift_{}_{}_body_adcode'.format(col,i)] = df.groupby('body_adcode')[col].shift(i)
            stat2_feat.append('shift_{}_{}_body_adcode'.format(col,i))
        df2 = df2.merge(df,'left',on = ['bodyType','adcode','mt','body_adcode_mt','body_adcode',col])
        df = df2.copy()
    
    return df2,stat2_feat

In [27]:
def get_stat_feature3(df_):  #省份
    df = df_.copy()
    print(df.shape)
    stat3_feat = []
#     df['adcode'] = df['adcode'] + df['bodyType']
    df['adcode_mt'] = df['adcode'] * 100 + df['mt']
    df2 = df.copy()
    for col in tqdm(['label','popularity','newsReplyVolum']):
        df = df.groupby(['adcode','mt','adcode_mt'])[col].mean().reset_index()
        for i in [1,2,3,4,5,6,7,8,9,10,11]:
            df['shift_{}_{}_adcode'.format(col,i)] = df.groupby('adcode')[col].shift(i)
            stat3_feat.append('shift_{}_{}_adcode'.format(col,i))
        df2 = df2.merge(df,'left',on = ['adcode','mt','adcode_mt',col])
        df = df2.copy()
    return df2,stat3_feat


In [28]:
def valid_score(data, pred='pred_label', label='label', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

In [29]:
def online_score(data, pred='pred_label', label='true_sales', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    y = pd.read_csv("valid_data_y.csv")
    data = data.merge(y, 'left', on=['id'])
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

In [30]:
def get_model_type(train_x,train_y,valid_x,valid_y,m_type='lgb'):   
    if m_type == 'lgb':
        model = lgb.LGBMRegressor(
                                num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
                                max_depth=-1, learning_rate=0.01, min_child_samples=1, random_state=1024,
                                n_estimators=2000, subsample=0.9, colsample_bytree=0.7,
                                )
        model.fit(train_x, train_y, 
                  
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              categorical_feature=cate_feat, 
              early_stopping_rounds=100, verbose=100)      
    elif m_type == 'xgb':
        model = xgb.XGBRegressor(
                                max_depth=5 , learning_rate=0.01, n_estimators=4000, 
                                objective='reg:gamma', tree_method = 'hist',subsample=0.9, 
                                colsample_bytree=0.7, min_child_samples=5,eval_metric = 'rmse' 
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              early_stopping_rounds=100, verbose=100)   
    return model

In [31]:
def get_train_model(df_, m, m_type):
    df = df_.copy()
    # 数据集划分
    st = 8
    all_idx   = (df['mt'].between(st , m-1))
    train_idx = (df['mt'].between(st , m-6))
    valid_idx = (df['mt'].between(m-5, m-5))
    test_idx  = (df['mt'].between(m  , m  ))
    
    print('all_idx  :',st ,m-1)
    print('train_idx:',st ,m-5)
    
        
    print('valid_idx:',m-4,m-4)
    print('test_idx :',m  ,m  )  
    # 最终确认
    train_x = df[train_idx][features]
    train_y = df[train_idx]['label']
    valid_x = df[valid_idx][features]
    valid_y = df[valid_idx]['label']   
    # get model
    model = get_model_type(train_x,train_y,valid_x,valid_y,m_type)  
    # offline
    df['pred_label'] = model.predict(df[features])
    best_score = valid_score(df[valid_idx]) 
    # online
    if m_type == 'lgb':
        model.n_estimators = model.best_iteration_ + 100
        model.fit(df[all_idx][features], df[all_idx]['label'], categorical_feature=cate_feat)
    elif m_type == 'xgb':
        model.n_estimators = model.best_iteration + 100
        model.fit(df[all_idx][features], df[all_idx]['label'])
    df['forecastVolum'] = model.predict(df[features])
    print('valid mean:',df[valid_idx]['pred_label'].mean())
    print('true  mean:',df[valid_idx]['label'].mean())
    print('test  mean:',df[test_idx]['forecastVolum'].mean())
    print("--------------------------")
    # 阶段结果
    sub = df[test_idx][['id']]
    sub['forecastVolum'] = df[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    print(sub.shape)
    return sub,df[valid_idx]['pred_label']

In [32]:
for month in [25,26,27,28]: 
#     judge = month-20
    m_type = 'lgb'
    data = Cluster(data,month)
    data_df = windows_feature(data)
    data_df2, stat_feat = get_stat_feature(data_df)
    data_df3, stat2_feat = get_stat_feature2(data_df2)
    data_df4, stat3_feat = get_stat_feature3(data_df3)

    win_feat = ['windows3_var','windows5_var','windows7_var']
#     winPop_feat = ['windowsPop4_var','windowsPop4_std','windowsPop5_var','windowsPop5_std','windowsPop12_var','windowsPop12_std']
#     ,'windows6_var','windows6_std']
#     qushi_feat = ['ratio2','differ2']
    num_feat = ['regYear','regMonth'] + stat_feat+stat2_feat+stat3_feat +win_feat#,'windows_var','windows_std'
    cate_feat = ['adcode','bodyType','model','model_adcode','body_adcode','cluster_label']

    if m_type == 'lgb':
        for i in cate_feat:
            data_df4[i] = data_df4[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))

    features = num_feat + cate_feat
    print("xxxxxxxxxxxxxxxxxx")
    print(len(features), len(set(features)))
    sub,val_pred = get_train_model(data_df4, month, m_type)
#     online_test_idx  = (data_df4['mt'].between(21  , 24  ))
    print("---")
    print(data.loc[(data.regMonth==month)].shape)
    data.loc[(data.mt==month), 'salesVolume'   ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'label'      ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'pred_label'      ] = sub['forecastVolum'].values
#     data['n_label'] = data['label'] / data['model_weight']

    # # sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
# # sub.columns = ['id','forecastVolum']
# # sub[['id','forecastVolum']].round().astype(int).to_csv('submit/yulao_lgb.csv', index=False)
# best_score = online_score(data[online_test_idx]) 

windows_var_features start


100%|██████████████████████████████████████████████████████████████████████████████| 1320/1320 [00:17<00:00, 73.94it/s]


windows_var_features end


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.27it/s]


(34320, 87)


...

(34320, 87)


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.81it/s]


(34320, 122)


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.12it/s]


xxxxxxxxxxxxxxxxxx
110 110
all_idx  : 8 27
train_idx: 8 23
valid_idx: 24 24
test_idx : 28 28
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 101390	valid_1's l2: 156231
[200]	training's l2: 31561.2	valid_1's l2: 57376.8
[300]	training's l2: 17624.9	valid_1's l2: 37763.2
[400]	training's l2: 12947.8	valid_1's l2: 33016.2
[500]	training's l2: 10370.8	valid_1's l2: 31282.4
[600]	training's l2: 8634.07	valid_1's l2: 30444.9
[700]	training's l2: 7453.23	valid_1's l2: 29936.5
[800]	training's l2: 6568.3	valid_1's l2: 29704.4
[900]	training's l2: 5906.41	valid_1's l2: 29521.7
Early stopping, best iteration is:
[876]	training's l2: 6041	valid_1's l2: 29449.2
0.7984281074939382
valid mean: 672.7943300750736
true  mean: 673.0143939393939
test  mean: 462.04289110920314
--------------------------
(1320, 2)
---
(0, 16)


In [34]:
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
print(sub.head())
print(sub.tail())

       id  forecastVolum
31680   1          267.0
31681   2          306.0
31682   3          176.0
31683   4          320.0
31684   5          333.0
         id  forecastVolum
36955  5364           55.0
36956  5365           55.0
36957  5366           77.0
36958  5367          198.0
36959  5368           45.0


In [12]:
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
print(sub.head())
print(sub.tail())

       id  forecastVolum
31680   1          279.0
31681   2          312.0
31682   3          161.0
31683   4          295.0
31684   5          352.0
         id  forecastVolum
36955  5364          100.0
36956  5365           94.0
36957  5366           95.0
36958  5367          213.0
36959  5368           69.0


In [35]:
sub[['id','forecastVolum']].round().astype(int).to_csv('submit/lgb_no_12.csv', index=False)

In [113]:
for month in [21,22,23,24]: 
    m_type = 'lgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear','regMonth'] + stat_feat
    cate_feat = ['adcode','bodyType','model']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
#     print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.mt==month), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.mt==month), 'label'      ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'pred_label'      ] = sub['forecastVolum'].values
    
    
    
    
    
# sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
# sub.columns = ['id','forecastVolum']
# sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.07it/s]


all_idx  : 9 20
train_idx: 9 16
valid_idx: 17 17
test_idx : 21 21
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 128712	valid_1's l2: 98700.9
[200]	training's l2: 41958.5	valid_1's l2: 38841.5
[300]	training's l2: 23711.2	valid_1's l2: 28937.2
[400]	training's l2: 17316.6	valid_1's l2: 26511.4
[500]	training's l2: 13789.8	valid_1's l2: 25838.3
[600]	training's l2: 11344.6	valid_1's l2: 25413.5
[700]	training's l2: 9716.53	valid_1's l2: 25005.3
[800]	training's l2: 8521.85	valid_1's l2: 24786.1
[900]	training's l2: 7601.14	valid_1's l2: 24786.9
[1000]	training's l2: 6807.82	valid_1's l2: 24710.5
[1100]	training's l2: 6154.66	valid_1's l2: 24588.3
[1200]	training's l2: 5603.01	valid_1's l2: 24436.2
[1300]	training's l2: 5128.15	valid_1's l2: 24299.6
[1400]	training's l2: 4756.47	valid_1's l2: 24098.7
[1500]	training's l2: 4411.44	valid_1's l2: 23997.3
[1600]	training's l2: 4081.94	valid_1's l2: 23915.1
[1700]	training's l2: 3801.56	valid_1's l2: 23785

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.20it/s]


all_idx  : 9 21
train_idx: 9 17
valid_idx: 18 18
test_idx : 22 22
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 124896	valid_1's l2: 101622
[200]	training's l2: 41221.5	valid_1's l2: 40668.4
[300]	training's l2: 23551.2	valid_1's l2: 30796.4
[400]	training's l2: 17247.5	valid_1's l2: 28629.1
[500]	training's l2: 13644.1	valid_1's l2: 28447.5
[600]	training's l2: 11311.3	valid_1's l2: 28214.8
[700]	training's l2: 9665.8	valid_1's l2: 28306.6
Early stopping, best iteration is:
[602]	training's l2: 11275.5	valid_1's l2: 28202.4
0.7483585609319627
valid mean: 547.3275192104005
true  mean: 523.3159090909091
test  mean: 648.8015280661291
(1320, 2)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.75it/s]


all_idx  : 9 22
train_idx: 9 18
valid_idx: 19 19
test_idx : 23 23
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 121129	valid_1's l2: 101400
[200]	training's l2: 40304.7	valid_1's l2: 38675.6
[300]	training's l2: 23218.8	valid_1's l2: 27340.3
[400]	training's l2: 17179.7	valid_1's l2: 24853.9
[500]	training's l2: 13713.1	valid_1's l2: 23959.2
[600]	training's l2: 11471.2	valid_1's l2: 23400.8
[700]	training's l2: 9962.11	valid_1's l2: 22931.2
[800]	training's l2: 8881.21	valid_1's l2: 22567.1
[900]	training's l2: 8007.93	valid_1's l2: 22383.6
[1000]	training's l2: 7299.37	valid_1's l2: 22289.3
[1100]	training's l2: 6668.06	valid_1's l2: 22270.3
[1200]	training's l2: 6149.12	valid_1's l2: 22150.4
Early stopping, best iteration is:
[1196]	training's l2: 6170.05	valid_1's l2: 22128.1
0.7468931536269028
valid mean: 560.7094271141314
true  mean: 552.2295454545455
test  mean: 734.8046071367794
(1320, 2)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.18it/s]


all_idx  : 9 23
train_idx: 9 19
valid_idx: 20 20
test_idx : 24 24
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 118772	valid_1's l2: 145395
[200]	training's l2: 39386.3	valid_1's l2: 51071.7
[300]	training's l2: 22711.1	valid_1's l2: 32006.2
[400]	training's l2: 17040	valid_1's l2: 27020.4
[500]	training's l2: 13774.6	valid_1's l2: 25125.4
[600]	training's l2: 11675.5	valid_1's l2: 24421
[700]	training's l2: 10208	valid_1's l2: 24129.7
[800]	training's l2: 9205.22	valid_1's l2: 23964.6
[900]	training's l2: 8388.5	valid_1's l2: 23799.9
[1000]	training's l2: 7727.56	valid_1's l2: 23768.7
Early stopping, best iteration is:
[989]	training's l2: 7801.96	valid_1's l2: 23734.8
0.7917019686244774
valid mean: 595.4352742803294
true  mean: 620.0212121212121
test  mean: 856.408786058386
(1320, 2)


In [68]:
for month in [21,22,23,24]: 
    m_type = 'lgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear','regMonth','popularity'] + stat_feat
    cate_feat = ['adcode','bodyType','model']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
#     print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values
#     data.loc[(data.mt==(month)), 'pred_label'      ] = sub['forecastVolum'].values
    
    
    
    
    
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
# sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.37it/s]


all_idx  : 13 24
train_idx: 13 20
valid_idx: 21 21
test_idx : 25 25
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 86584.9	valid_1's l2: 151719
[200]	training's l2: 27784.3	valid_1's l2: 71704.5
[300]	training's l2: 14902.5	valid_1's l2: 57361.3
[400]	training's l2: 10243.5	valid_1's l2: 52224.7
[500]	training's l2: 7869.38	valid_1's l2: 50304.7
[600]	training's l2: 6385.06	valid_1's l2: 49569.7
[700]	training's l2: 5380.65	valid_1's l2: 48911.9
[800]	training's l2: 4634.41	valid_1's l2: 48274
[900]	training's l2: 4071.47	valid_1's l2: 48026.2
[1000]	training's l2: 3625	valid_1's l2: 47641.9
[1100]	training's l2: 3253.2	valid_1's l2: 47425.4
[1200]	training's l2: 2928.61	valid_1's l2: 47143.3
[1300]	training's l2: 2653.04	valid_1's l2: 46870.8
[1400]	training's l2: 2414.07	valid_1's l2: 46669.9
[1500]	training's l2: 2207.68	valid_1's l2: 46487.6
[1600]	training's l2: 2028.06	valid_1's l2: 46336.7
[1700]	training's l2: 1876.96	valid_1's l2: 46208.4
[

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.41it/s]


all_idx  : 13 25
train_idx: 13 21
valid_idx: 22 22
test_idx : 26 26
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 90083.5	valid_1's l2: 118412
[200]	training's l2: 28475.8	valid_1's l2: 59906.1
[300]	training's l2: 15309.6	valid_1's l2: 52080.3
[400]	training's l2: 10785.7	valid_1's l2: 50404.3
[500]	training's l2: 8324.54	valid_1's l2: 49632.1
[600]	training's l2: 6828.3	valid_1's l2: 49046.4
[700]	training's l2: 5813.9	valid_1's l2: 48997.8
Early stopping, best iteration is:
[648]	training's l2: 6301.83	valid_1's l2: 48762.4
0.7352932759641437


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.40it/s]


all_idx  : 13 26
train_idx: 13 22
valid_idx: 23 23
test_idx : 27 27
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 93591.6	valid_1's l2: 135289
[200]	training's l2: 29582.9	valid_1's l2: 52574.8
[300]	training's l2: 15864.8	valid_1's l2: 39181.2
[400]	training's l2: 11154.7	valid_1's l2: 36023.5
[500]	training's l2: 8739.67	valid_1's l2: 35331.5
[600]	training's l2: 7173.39	valid_1's l2: 34526.5
[700]	training's l2: 6126.06	valid_1's l2: 34402.5
[800]	training's l2: 5347.97	valid_1's l2: 34349
[900]	training's l2: 4770.82	valid_1's l2: 34426
Early stopping, best iteration is:
[806]	training's l2: 5308.31	valid_1's l2: 34325.1
0.7811424903161602


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.38it/s]


all_idx  : 13 27
train_idx: 13 23
valid_idx: 24 24
test_idx : 28 28
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 96023.9	valid_1's l2: 552065
[200]	training's l2: 29709.3	valid_1's l2: 354551
[300]	training's l2: 16067.9	valid_1's l2: 299015
[400]	training's l2: 11490.5	valid_1's l2: 282547
[500]	training's l2: 9067.98	valid_1's l2: 274322
[600]	training's l2: 7522.51	valid_1's l2: 271372
[700]	training's l2: 6477.28	valid_1's l2: 271395
Early stopping, best iteration is:
[658]	training's l2: 6879.88	valid_1's l2: 270441
0.6226761089064834


In [69]:
sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales0914.csv', index=False)

In [38]:
for month in [25,26,27,28]: 
    m_type = 'lgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear'] + stat_feat
    cate_feat = ['adcode','bodyType','model','regMonth']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
    print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
# sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.38s/it]


53 53
all_idx  : 13 24
train_idx: 13 20
valid_idx: 21 21
test_idx : 25 25
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 7291.26	valid_1's l2: 37932.3
[200]	training's l2: 3715.93	valid_1's l2: 35716.9
[300]	training's l2: 2515.27	valid_1's l2: 35194.8
[400]	training's l2: 1831.38	valid_1's l2: 34788.8
[500]	training's l2: 1397.64	valid_1's l2: 34608.6
[600]	training's l2: 1065.51	valid_1's l2: 34527.1
[700]	training's l2: 849.095	valid_1's l2: 34415.2
[800]	training's l2: 679.238	valid_1's l2: 34333.1
[900]	training's l2: 545.247	valid_1's l2: 34326.4
Early stopping, best iteration is:
[838]	training's l2: 624.086	valid_1's l2: 34316.6
0.7559303097480222
valid mean: 602.8972278482273
true  mean: 649.3121212121212
test  mean: 493.9801201875369


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.35s/it]


53 53
all_idx  : 13 25
train_idx: 13 21
valid_idx: 22 22
test_idx : 26 26
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 7936.91	valid_1's l2: 41558.5
Early stopping, best iteration is:
[75]	training's l2: 10761.5	valid_1's l2: 41117.3
0.7414955122487169
valid mean: 623.4500376146684
true  mean: 616.5537878787878
test  mean: 324.6451813135614


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.36s/it]


53 53
all_idx  : 13 26
train_idx: 13 22
valid_idx: 23 23
test_idx : 27 27
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 8653.84	valid_1's l2: 32560.4
[200]	training's l2: 4627.91	valid_1's l2: 32133.2
Early stopping, best iteration is:
[155]	training's l2: 5757.15	valid_1's l2: 31853.4
0.7794578141087838
valid mean: 639.6227869473659
true  mean: 673.0143939393939
test  mean: 479.55100257280156


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.40s/it]


53 53
all_idx  : 13 27
train_idx: 13 23
valid_idx: 24 24
test_idx : 28 28
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 9306.24	valid_1's l2: 302435
[200]	training's l2: 5107.21	valid_1's l2: 292129
[300]	training's l2: 3583.18	valid_1's l2: 289373
[400]	training's l2: 2709.06	valid_1's l2: 288389
[500]	training's l2: 2162.77	valid_1's l2: 287983
[600]	training's l2: 1746.75	valid_1's l2: 287504
[700]	training's l2: 1446.22	valid_1's l2: 287029
[800]	training's l2: 1196.94	valid_1's l2: 286429
[900]	training's l2: 1008.26	valid_1's l2: 286030
Early stopping, best iteration is:
[856]	training's l2: 1089.91	valid_1's l2: 286004
0.588139278724323
valid mean: 645.3917897243205
true  mean: 899.8204545454546
test  mean: 468.0356962498267
